In [1]:
import pandas as pd
import re

In [40]:
types = pd.read_sql_table('element_types', "sqlite:///database.db", index_col='id')
types

,name
id,
1,definition
2,example
3,theorem
4,figure
5,condition
6,exercise
7,axiom
8,table
9,property


In [41]:
elements = pd.read_sql_table('elements', "sqlite:///database.db", index_col='id')
elements.head()

,subsection_id,type_id,number,content
id,,,,
1,1,1,1,**Median.** Let $X$ be a random variable. Ever...
2,1,2,1,**The Median of a Discrete Distribution.** Sup...
3,1,2,2,**A Discrete Distribution for Which the Median...
4,1,2,3,**The Median of a Continuous Distribution.** S...
5,1,2,4,**A Continuous Distribution for Which the Medi...


In [46]:
type_names = "|".join(types.name.str.lower().tolist())
elements.content.str.extract(rf'(?P<type>{type_names})\s+(?P<number>[\d\.]+)', flags=re.IGNORECASE)

,type,number
id,,
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
...,...,...
4243,Exercise,11
4244,Exercise,1
4245,Exercise,13


In [58]:
elements[elements.content.str.contains('exercise.*sec', case=False)]

,subsection_id,type_id,number,content,match_type,match_number
id,,,,,,
82,3,6,9,9. Suppose that a certain system contains thre...,exercise,5
106,4,2,8,### The Effect of Prior Probabilities.\nConsid...,example,2.3.7.
115,4,6,9,9. Consider again the conditions of Exercise 1...,exercise,14
120,4,6,14,14. Consider again the conditions of Exercise ...,exercise,23
164,6,2,10,**Sampling from a Mixture of Two Distributions...,exercise,10
...,...,...,...,...,...,...
4232,2,13,1,For the random number generator that I have be...,exercise,1
4233,2,13,2,The plots for this exercise are formed the sam...,exercise,1
4234,2,13,3,"Once again, the plots are drawn in a fashion s...",exercise,1.


In [90]:
elements.content.str.extractall(r'(?P<type1>exercise)\s+(?P<number1>\d+)(?:[\w\s]{1,6}(?P<type2>sec\.|section|exercise)\s*(?P<number2>\d+))?', flags=re.IGNORECASE)

type1 number1 type2 number2
id   match                                
24   0      Exercise       9   NaN     NaN
45   0      Exercise      11   NaN     NaN
     1      Exercise      11   NaN     NaN
47   0      Exercise      13   NaN     NaN
     1      Exercise      13   NaN     NaN
...              ...     ...   ...     ...
4239 0      Exercise       9   NaN     NaN
4242 0      Exercise      12  Sec.      12
4243 0      Exercise      11   NaN     NaN
4244 0      Exercise       1  Sec.       8
4245 0      Exercise      13   NaN     NaN

[768 rows x 4 columns]

In [89]:
elements.content.str.extractall(r'(?P<type1>exercise)\s+(?P<number1>\d+)[\w\s]{1,6}(?P<type2>sec\.|section|exercise)\s*(?P<number2>\d+)', flags=re.IGNORECASE)

,,type1,number1,type2,number2
id,match,,,,
82,0,Exercise,5,Sec.,3
115,0,Exercise,14,Sec.,2
120,0,Exercise,23,Sec.,2
164,0,Exercise,10,Sec.,12
237,0,Exercise,24,Sec.,7
...,...,...,...,...,...
4232,0,Exercise,1,Sec.,12
4233,0,Exercise,2,Sec.,12
4234,0,Exercise,3,Sec.,12


In [75]:
type_names = "|".join(types.name.str.lower().tolist())
elements.content.str.extractall(rf'(?P<type>{type_names})\s+(?P<number>[\d\.]+)', flags=re.IGNORECASE)

type  number
id   match                  
6    0       Example   4.5.5
     1       Example   4.3.5
     2       Example   4.3.5
11   0       Example   4.5.5
     1       Example   4.5.5
...              ...     ...
4242 0      Exercise      12
4243 0      Exercise      11
4244 0      Exercise       1
4245 0      Exercise      13
4247 0       Example  12.5.2

[2583 rows x 2 columns]

In [67]:
type_names = "|".join(types.name.str.lower().tolist())
elements.content.str.extractall(
    rf'(?P<type1>{type_names}|sec\.|section)\s+'
    rf'(?P<number1>[\d.]+)'
    rf'[\w\s]{{0,6}}'
    rf'(?P<type2>sec\.|section|exercise)?\s*'
    rf'(?P<number2>\d+)?',
    flags=re.IGNORECASE
)

type1 number1 type2 number2
id   match                                
6    0       Example   4.5.5   NaN     NaN
     1       Example   4.3.5   NaN     NaN
     2       Example   4.3.5   NaN     NaN
11   0       Example   4.5.5   NaN     NaN
     1       Example   4.5.5   NaN     NaN
...              ...     ...   ...     ...
4242 0      Exercise      12   NaN     NaN
4243 0      Exercise      11   NaN     NaN
4244 0      Exercise       1   NaN     NaN
4245 0      Exercise      13   NaN     NaN
4247 0       Example  12.5.2   NaN     NaN

[2702 rows x 4 columns]

In [99]:
type_names = "|".join(types.name.str.lower().tolist())
matches = elements.content.str.extractall(
    rf'(?P<type1>{type_names}|sec\.|section)\s+'
    rf'(?P<number1>[\d.]+)'
    rf'(?:[\w\s]{{1,6}}(?P<type2>sec\.|section|exercise)\s*(?P<number2>\d+))?',
    flags=re.IGNORECASE
).reset_index(drop=False)

matches

,id,match,type1,number1,type2,number2
0,6,0,Example,4.5.5,NaN,NaN
1,6,1,Example,4.3.5,NaN,NaN
2,6,2,Example,4.3.5,NaN,NaN
3,11,0,Example,4.5.5,NaN,NaN
4,11,1,Example,4.5.5,NaN,NaN
...,...,...,...,...,...,...
2704,4242,0,Exercise,12,Sec.,12
2705,4243,0,Exercise,11,NaN,NaN
2706,4244,0,Exercise,1,Sec.,8
2707,4245,0,Exercise,13,NaN,NaN


In [101]:
matches['type2'].unique()

array([nan, 'Sec.', 'Section', 'Exercise'], dtype=object)

In [102]:
matches[matches['type2'].isin(['Section', 'Exercise'])]

,id,match,type1,number1,type2,number2
238,316,0,Exercise,5,Section,6
265,361,1,Exercise,13,Section,1
589,744,0,Exercise,9,Section,7
1102,1455,0,Example,9.1.17,Exercise,19
2323,3551,0,Exercise,11,Exercise,24
2368,3602,1,Example,7.7.5,Exercise,12


In [48]:
elements[["match_type", "match_number"]] = elements.content.str.extract(rf'(?P<match_type>{type_names})\s+(?P<match_number>[\d\.]+)', flags=re.IGNORECASE)
elements["match_type"] = elements["match_type"].str.lower()
elements

,subsection_id,type_id,number,content,match_type,match_number
id,,,,,,
1,1,1,1,**Median.** Let $X$ be a random variable. Ever...,NaN,NaN
2,1,2,1,**The Median of a Discrete Distribution.** Sup...,NaN,NaN
3,1,2,2,**A Discrete Distribution for Which the Median...,NaN,NaN
4,1,2,3,**The Median of a Continuous Distribution.** S...,NaN,NaN
5,1,2,4,**A Continuous Distribution for Which the Medi...,NaN,NaN
...,...,...,...,...,...,...
4243,2,13,12,I used the same simulations as in Exercise 11 ...,exercise,11
4244,2,13,13,(a) The fact that $E\left(\hat{\beta}_{1}\righ...,exercise,1
4245,2,13,14,"As we noted in Exercise 13(c), the value of $v...",exercise,13


In [ ]:
df["example_match"] = df.content.str.extract(r'(example [\d\.]+)', flags=re.IGNORECASE)
df[df.example_match.notnull()]

,subsection_id,type_id,number,content,example_match
id,,,,,
6,1,2,5,**Last Lottery Number.** In a state lottery ga...,Example 4.5.5
11,1,2,7,"**Last Lottery Number.** In Example 4.5.5, we ...",Example 4.5.5
14,1,2,8,"**Last Lottery Number.** In Example 4.5.5, in ...",Example 4.5.5
29,1,6,14,14. Find the median of the Cauchy distribution...,Example 4.1.8.
38,2,6,5,5. Consider the power calculation done in Exam...,Example 9.5.5.
...,...,...,...,...,...
4226,8,13,7,(a) Each bootstrap sample consists of $\bar{X}...,Example 12.6.10
4237,2,13,6,"(a) For each sample, we compute the numbers of...",Example 10.1.6.
4239,2,13,8,(a) The product of likelihood times prior is\n...,Example 12.5.6


In [37]:
df[(df.type_id == 6) & (df.content.str.contains('example [\d\.]+', case=False))]

,subsection_id,type_id,number,content,example_match
id,,,,,
29,1,6,14,14. Find the median of the Cauchy distribution...,Example 4.1.8.
38,2,6,5,5. Consider the power calculation done in Exam...,Example 9.5.5.
39,2,6,6,6. The $\chi^2$ goodness-of-fit test (see Chap...,Example 10.1.6.
40,2,6,7,"7. In Sec. 10.2, we discussed $\chi^2$ goodnes...",Example 10.2.5
41,2,6,8,"8. In Example 12.5.6, we used a hierarchical m...",Example 12.5.6
...,...,...,...,...,...
2520,102,6,14,14. Suppose that we wish to integrate the same...,Example 12.4.1.
2522,102,6,16,16. Use the method of antithetic variates that...,Example 12.4.1.
2587,104,6,16,16. Consider the investor in Example 4.2.3 on ...,Example 4.2.3


In [ ]:
query = "SELECT s.section_id, s.number, e.* FROM elements AS e JOIN subsections AS s ON e.subsection_id = s.id"
join = pd.read_sql(query, "sqlite:///database.db", index_col='id')

join.head()

,subsection_id,type_id,number,content,section_id,number
id,,,,,,
1,1,1,1,**Median.** Let $X$ be a random variable. Ever...,4,5
2,1,2,1,**The Median of a Discrete Distribution.** Sup...,4,5
3,1,2,2,**A Discrete Distribution for Which the Median...,4,5
4,1,2,3,**The Median of a Continuous Distribution.** S...,4,5
5,1,2,4,**A Continuous Distribution for Which the Medi...,4,5
